<a href="https://colab.research.google.com/github/gargii21/MLPros/blob/main/Final_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nibabel
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [ ]:
# !pip install nibabel
# !pip install tensorflow
import os
import numpy as np
import pandas as pd
import nibabel as nib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize

In [ ]:
CSV_PATH = '/content/drive/MyDrive/ADNI1_Complete_3Yr_3T_7_04_2025.csv'
MRI_DIR = '/content/drive/MyDrive/ADNI_Imaging30to60'
TARGET_SHAPE = (155, 155, 95)

listOfFiles = list()

for (dirpath, dirnames, filenames) in os.walk(MRI_DIR):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]
# Load Metadata
df = pd.read_csv('/content/drive/MyDrive/ADNI1_Complete_3Yr_3T_7_04_2025 .csv')
print(df.head())
print(len(listOfFiles))
path=[]
for elem in listOfFiles:
  path.append(elem)

  Image Data ID     Subject Group Sex  Age Visit Modality  \
0        I83549  136_S_0429   MCI   M   64   m12      MRI   
1        I40392  136_S_0429   MCI   M   63    bl      MRI   
2       I120446  136_S_0429   MCI   M   65   m24      MRI   
3       I119733  136_S_0429   MCI   M   63    bl      MRI   
4        I88169  136_S_0429   MCI   M   64   m18      MRI   

           Description       Type    Acq Date Format Downloaded  
0    MPR; ; N3; Scaled  Processed   6/18/2007  NiFTI  7/04/2025  
1    MPR; ; N3; Scaled  Processed   6/27/2006  NiFTI  7/04/2025  
2    MPR; ; N3; Scaled  Processed   6/17/2008  NiFTI  7/04/2025  
3  MPR; ; N3; Scaled_2  Processed   6/27/2006  NiFTI  7/04/2025  
4    MPR; ; N3; Scaled  Processed  12/17/2007  NiFTI  7/04/2025  
161


In [ ]:
list1=[]
date_list=[]
sub_list=[]
for elem in listOfFiles:
  list1.append(elem)
for i in range(len(list1)):
  date_list.append('/'.join(list1[i].split('/')[7].split('_')[0].split('-')[1:] + [list1[i].split('/')[7].split('_')[0].split('-')[0]]))


  sub_list.append(list1[i].split('/')[5])

In [ ]:

file_content=list(zip(date_list,sub_list))
print(file_content[:10])

[('12/18/2006', '136_S_0429'), ('07/13/2009', '136_S_0429'), ('06/18/2007', '136_S_0429'), ('06/17/2008', '136_S_0429'), ('06/27/2006', '136_S_0429'), ('12/17/2007', '136_S_0429'), ('06/27/2006', '136_S_0429'), ('06/03/2008', '136_S_0426'), ('05/30/2006', '136_S_0426'), ('11/20/2006', '136_S_0426')]


In [ ]:
idx=pd.DataFrame(np.arange(0,351))
idx.columns=['Index']
idx

,Index
0,0
1,1
2,2
3,3
4,4
...,...
346,346
347,347
348,348
349,349


In [ ]:
total_data = (df.Group !=0).values.astype('bool')
metadata1=pd.concat([idx,df],axis=1)
metadata1[total_data]

,Index,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,0,I83549,136_S_0429,MCI,M,64,m12,MRI,MPR; ; N3; Scaled,Processed,6/18/2007,NiFTI,7/04/2025
1,1,I40392,136_S_0429,MCI,M,63,bl,MRI,MPR; ; N3; Scaled,Processed,6/27/2006,NiFTI,7/04/2025
2,2,I120446,136_S_0429,MCI,M,65,m24,MRI,MPR; ; N3; Scaled,Processed,6/17/2008,NiFTI,7/04/2025
3,3,I119733,136_S_0429,MCI,M,63,bl,MRI,MPR; ; N3; Scaled_2,Processed,6/27/2006,NiFTI,7/04/2025
4,4,I88169,136_S_0429,MCI,M,64,m18,MRI,MPR; ; N3; Scaled,Processed,12/17/2007,NiFTI,7/04/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,346,I64551,002_S_0413,CN,F,77,m12,MRI,MPR; ; N3; Scaled,Processed,6/01/2007,NiFTI,7/04/2025
347,347,I40657,002_S_0413,CN,F,76,bl,MRI,MPR; ; N3; Scaled,Processed,5/19/2006,NiFTI,7/04/2025
348,348,I118675,002_S_0413,CN,F,76,bl,MRI,MPR; ; N3; Scaled_2,Processed,5/19/2006,NiFTI,7/04/2025
349,349,I149735,002_S_0413,CN,F,79,m36,MRI,MPR; ; N3; Scaled,Processed,4/30/2009,NiFTI,7/04/2025


In [ ]:
labels = []
for elem in listOfFiles:
  split_names = elem.split('/')
  subject_name = split_names[5]

  indexM=metadata1.loc[metadata1['Subject']==(subject_name)]
  group=metadata1.Group[metadata1['Subject']==(subject_name)]

  group=pd.DataFrame(group)
  group = group.iloc[0]['Group']
  if group=='AD':
      label = 2

  if group=='CN':
      label = 0

  if group=='MCI':
      label = 1
  labels.append(label)


labels[:10]

[1, 1, 1, 1, 1, 1, 1, 2, 2, 2]

In [ ]:
# Load NIfTI files and labels
def load_mri_data(path, labels, target_shape):
    images = []
    for p in path:
        img = nib.load(p).get_fdata()
        img = resize(img, target_shape, mode='constant', preserve_range=True)
        img = (img - np.mean(img)) / (np.std(img) + 1e-8)  # Normalize
        images.append(img)
    return np.array(images), np.array(labels)

# Load data
X_test, y_test = load_mri_data(path, labels, TARGET_SHAPE)

# Add channel dimension for CNN input
X_test = X_test[..., np.newaxis]  # Shape: (N, D, H, W, 1)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y_test)
y_test_categorical = to_categorical(y_encoded)  # For softmax-compatible models

print(f"Test samples: {X_test.shape[0]}")
print(f"Classes: {le.classes_}")


Test samples: 161
Classes: [0 1 2]


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dense, Dropout
from keras.optimizers import Adam

input_shape = X_train.shape[1:]  # (D, H, W, 1)
num_classes = y_categorical.shape[1]

inputs = Input(shape=input_shape)

x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling3D((2, 2, 2))(x)

x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling3D((2, 2, 2))(x)

x = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling3D((2, 2, 2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


NameError: name 'X_train' is not defined

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/ADNI_model_1.h5')

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Model

# Create the feature extractor model
feature_extractor = Model(inputs=model.input,
                          outputs=model.get_layer('dense_1').output)

# Store results
features_list = []
file_content_list = []

# Loop through test data
for i in range(len(X_test)):
    sample = np.expand_dims(X_test[i], axis=0)
    features = feature_extractor.predict(sample)[0]  # Flatten output
    features_list.append(features)
    file_content_list.append(file_content[i])        # Tuple: (Subject_ID, Visit)

# Create the DataFrame
df = pd.DataFrame({
    'features': features_list,
    'file_content': file_content_list
})

# Show the DataFrame
print(df.head())


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [ ]:
df.head()

,features,file_content
0,"[38.911938, 0.0, 11.975995, 5.3666162, 10.4832...","(12/18/2006, 136_S_0429)"
1,"[37.245125, 0.0, 13.389341, 4.903385, 10.30850...","(07/13/2009, 136_S_0429)"
2,"[38.463264, 0.0, 12.580444, 5.4414415, 9.75999...","(06/18/2007, 136_S_0429)"
3,"[39.93951, 0.0, 16.722813, 0.0, 7.0721283, 0.0...","(06/17/2008, 136_S_0429)"
4,"[40.02019, 0.0, 12.433074, 5.1111836, 7.498178...","(06/27/2006, 136_S_0429)"


In [ ]:
df.to_csv('/content/drive/MyDrive/feature_vector30to60.csv', index=False)

In [ ]:
listOfFiles
subjects_count = 0
subjects_dict=[]
subjects_dict1 = []
for elem in listOfFiles:
    # a+=1
    # if a==3:
    #   break
    #im = 'ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070216232854688_S14782_I40657.nii'
    #img = nib.load(elem).get_data()
    #img = np.asarray(img)
    split_names = elem.split('/')
    subject_name = split_names[5]

    if subjects_count==0:
        subjects_count +=1
        subjects_dict.append(subject_name)
        subjects_dict1.append(subject_name)
    else:
        if subject_name in subjects_dict:
            subjects_count += 0
            #subjects_dict.append(subject_name)
	    #continue
        else:
            subjects_count += 1
            subjects_dict.append(subject_name)
            subjects_dict1.append(subject_name)
    if subjects_count>1072:
         break
np.shape(subjects_dict)
#subjects_dict1 = subjects_dict


(30,)

In [ ]:
gp = pd.read_csv('/content/drive/MyDrive/cleaned_data.csv')

In [ ]:
gp

,Grp,Sex,Totscore,Total13,DXNORM,DXMCI,DXAD,Acq_Date,Subject_ID,Visit,Subject_Visit
0,CN,F,3.33,4.33,1,0,0,5/19/2006,002_S_0413,bl,"('5/19/2006', '002_S_0413')"
1,CN,F,7.33,8.33,1,0,0,11/15/2006,002_S_0413,m06,"('11/15/2006', '002_S_0413')"
2,CN,F,2.33,4.33,1,0,0,06/01/2007,002_S_0413,m12,"('06/01/2007', '002_S_0413')"
3,CN,F,2.33,5.33,1,0,0,7/31/2008,002_S_0413,m24,"('7/31/2008', '002_S_0413')"
4,CN,F,1.33,4.33,1,0,0,4/30/2009,002_S_0413,m36,"('4/30/2009', '002_S_0413')"
...,...,...,...,...,...,...,...,...,...,...,...
296,MCI,M,13.33,20.33,0,1,0,12/18/2006,136_S_0429,m06,"('12/18/2006', '136_S_0429')"
297,MCI,M,15.33,22.33,0,0,1,6/18/2007,136_S_0429,m12,"('6/18/2007', '136_S_0429')"
298,MCI,M,19.00,26.00,0,1,0,12/17/2007,136_S_0429,m18,"('12/17/2007', '136_S_0429')"
299,MCI,M,15.67,25.67,0,1,0,6/17/2008,136_S_0429,m24,"('6/17/2008', '136_S_0429')"


In [ ]:
idx=0
for val in gp['Subject_ID']:
    if val not in subjects_dict:
      gp.drop(idx,inplace=True)
    idx+=1


In [ ]:
gp.to_csv('/content/drive/MyDrive/graphable_csv30to60.csv', index=False)